In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
#Preprocessing the training data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/tf/dataset/tomato/train',
                                                 target_size = (96, 96),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 10000 images belonging to 10 classes.


In [4]:
#Preprocessing the testing data
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/tf/dataset/tomato/val',
                                            target_size = (96, 96),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1000 images belonging to 10 classes.


In [8]:
#Initializing the CNN
cnn = tf.keras.models.Sequential()

#Convolution
cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 6, activation = 'relu', input_shape = [96, 96, 3]))

#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 4, strides = 3))

#Adding a second convolution and pooling layer
cnn.add(tf.keras.layers.Conv2D(filters = 56, kernel_size = 5, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 4, strides = 3))

#Flattening
cnn.add(tf.keras.layers.Flatten())

#Full Connection
cnn.add(tf.keras.layers.Dense(units = 512, activation = 'relu'))
cnn.add(tf.keras.layers.Dense(units = 1024, activation = 'relu'))
cnn.add(tf.keras.layers.Dense(units = 512, activation = 'relu'))
cnn.add(tf.keras.layers.Dense(units = 1024, activation = 'relu'))
cnn.add(tf.keras.layers.Dense(units = 512, activation = 'relu'))
    
#Output Layer
cnn.add(tf.keras.layers.Dense(units = 10, activation = 'sigmoid'))
    
#Training the CNN
    
#Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

#Training the CNN on the training set and testing it on the testing dataset
with tf.device('/gpu:0'):
    cnn.fit(x = training_set, validation_data = test_set, epochs = 50, verbose = 1)

Epoch 1/50
209/209 [==============================] - 41s 193ms/step - loss: 2.3032 - accuracy: 0.1017 - val_loss: 2.3027 - val_accuracy: 0.1000
Epoch 2/50
209/209 [==============================] - 41s 196ms/step - loss: 2.2976 - accuracy: 0.1042 - val_loss: 2.2896 - val_accuracy: 0.1060
Epoch 3/50
209/209 [==============================] - 41s 198ms/step - loss: 2.2503 - accuracy: 0.1317 - val_loss: 2.1779 - val_accuracy: 0.1630
Epoch 4/50
209/209 [==============================] - 40s 192ms/step - loss: 2.1001 - accuracy: 0.2114 - val_loss: 2.1572 - val_accuracy: 0.1720
Epoch 5/50
209/209 [==============================] - 41s 194ms/step - loss: 1.8918 - accuracy: 0.2924 - val_loss: 1.9443 - val_accuracy: 0.3090
Epoch 6/50
209/209 [==============================] - 40s 192ms/step - loss: 1.6856 - accuracy: 0.3676 - val_loss: 1.5649 - val_accuracy: 0.4340
Epoch 7/50
209/209 [==============================] - 40s 193ms/step - loss: 1.5136 - accuracy: 0.4468 - val_loss: 1.4635 - val_ac

In [10]:
from tensorflow.keras.models import load_model
cnn.save('model_weights.h5')